# Measuring SNR of Target

## 信噪比

* 我们定义信噪比(Signal to Noise Ratio)为

    $$
    SNR = \frac{Var(Signal)}{Var(Noise)}
    $$

    同时我们经常使用$dB$来表达SNR，有

    $$
    SNR_{dB} = 20\log{(SNR)}
    $$

    这里的20是因为假设我们测量的是电压，那么电阻两端的功率应该是电压的平方，所以有

    $$
    SNR_{dB} = 10\log{\left((\frac{Var(Signal)}{Var(Noise)})^2\right)}
    $$

## 什么是信号

* 信号是我们基于泄漏函数捕获的信息，噪音则是其固有的噪音而不是因泄漏引起的。

* 我们要做的就是找到泄漏信息中的平均轨迹。如果使用海明重量模型的话我们可以得到9组轨迹。如果使用传统DPA则可以得到两组。

* 对于每一组，我们都可以来测量它的噪音，但我们不需要测量所有组，因为我们只需要测量噪音，而不是信号的方差。

## 计算信噪比

* （Xenny）这里测量的是随机1000组随机明文和密钥加密轨迹，然后我们的海明重量模型是基于第一组S盒置换。此时明文和密钥发生异或并进行S盒变换，计算此时的海明重量。至于为什么要计算此时的海明重量，因为本篇只涉及计算信噪比，在后面的AES差分功耗分析中会再次讲解。

* 要做的第一件事是传播这些轨迹，然后取出每个组的平均值记录到数组中。

In [ ]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]

# SNR of byte 0
bnum = 0
npoints = len(traces[0].wave)
hwarray = [[] for i in range(9)]
for tnum in range(0, len(traces)):
    hw_of_byte = HW[intermediate(traces[tnum].textin[bnum], traces[tnum].key[bnum])]
    hwarray[hw_of_byte].append(traces[tnums].wave)

hwmean = np.zeros((9, npoints))
for i in range(0, 9):
    hwmean[i] = np.mean(hwarray[i], axis=0)

* 然后画图，每一组轨迹中的数字不会是统一的。因为大部分的值会是`HW(4)`。类似正态分布。

* 然后我们以`hwarray[4]`为例计算噪音，记住需要先移除零轨迹，因为它会扰乱方差

In [ ]:
inc_list = []
for i in range(0, len(hwarray)):
    if len(hwarray[i]) > 0:
        inc_list.append(i)
        
hwmean_valid = hwmean[inc_list]

signal_var = np.var(hwmean_valid, axis=0)
noise_var_onehw = np.var(hwarray[4], axis=0)

snr = signal_var / noise_var_onehw
hv.Curve(20 * np.log(snr)).opts(height=600, width=600)

## 自动SNR绘制

* CW集成了计算SNR的代码，基于一个泄漏模型即可轻松计算。

In [ ]:
import chipwhisperer.analyzer as cwa

leak_model = cwa.leakage_models.sbox_output

snrdb = cwa.calculate_snr(traces, leak_model=leak_model)

hv.Curve(snrdb).opts(height=600, width=600)